Project 1: Writing a Data Scientist Blog Post

Section 1: Business Understanding

This notebook will analyze Boston Airbnb Data to answer the following three questions:
1. How does seasonal variation impact Boston AirBnB booking prices?
2. What are the most common amenities for Boston AirBnB properties?
3. Which neighborhood has the most AirBnB properties and how does that correlate to pricing?

Import required packages:

In [436]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

Section 2: Data Understanding

Get familiar with the contents of the calendar file.

In [ ]:
# Open calendar data file
calendar_path = r"C:\Users\emma1\Project_1_Data-Science-Blog\Boston Airbnb Open Data\calendar.csv"
calendar = pd.read_csv(calendar_path)
# View top five rows
calendar.head()

In [ ]:
# How big is the dataset?
calendar.shape # 4 columns : 1,308,890 rows of data

In [ ]:
# What data types are included in this dataset?
calendar.dtypes # Why is price listed as an object?

In [ ]:
# Count null values
calendar.isnull().sum() # 665,853 null values in price column

In [ ]:
# Determine proportion of null values
calendar.isnull().mean() # 50.8716% null values in price column

In [ ]:
# Look at descriptive statistics for each field
calendar.describe(include='all') # statistics not very useful - only listing_id is an int field, others are object fields

Get familiar with reviews file.

In [ ]:
# Import reviews dataset
reviews_path = r"C:\Users\emma1\Project_1_Data-Science-Blog\Boston Airbnb Open Data\reviews.csv"
reviews = pd.read_csv(reviews_path)

# View top five rows
print(reviews.head())

In [ ]:
# How big is this dataset?
reviews.shape # 6 columns : 68,275 rows

In [ ]:
reviews.value_counts()

In [ ]:
reviews.isnull().mean()
reviews.isnull().sum()
# 53 rows missing comments - no way to impute values

In [ ]:
reviews['reviewer_id'].value_counts()
# not sure what to do with this information...looks like the listings file will provide more insight

Get familiar with the listings file. 

In [ ]:
# Open listings data file
listings_path = r"C:\Users\emma1\Project_1_Data-Science-Blog\Boston Airbnb Open Data\listings.csv"
listings = pd.read_csv(listings_path)
# View top five rows
print(listings.head()) # lots of data here

In [ ]:
# How big is this dataset?
listings.shape # 95 columns : 3,585 rows

In [ ]:
listings.columns # look at column names

In [ ]:
listings.describe(include='all')

In [ ]:
# How many nulls? What proportion of nulls? What are the column types?
for column in listings.columns:
    num_null = listings[column].isnull().sum()
    prop_null = listings[column].isnull().mean()
    col_type = listings[column].dtype
    print(column + ":" + str(num_null) + "," + str(prop_null) + "," + str(col_type))

Section 3: Data Preparation

Clean up calendar file.

In [ ]:
# Change date field from object to datetime
calendar['date'] = pd.to_datetime(calendar['date'])
calendar.dtypes # confirm change was successful

In [ ]:
# price field has object dtype due to $ character
# Remove $ from beginning of each string
calendar['price'] = calendar['price'].str.replace("$", "")
calendar['price'].value_counts()

In [ ]:
# Convert price field to numeric
calendar['price'] = pd.to_numeric(calendar['price'], errors= "coerce")
calendar.dtypes # confirm change was successful

Visualize the cleaned calendar data.

In [ ]:
plt.hist(calendar['price']);
# right skewed distribution

In [ ]:
plt.hist(calendar['available']);
# Fairly equal distribution of T and F values

In [ ]:
plt.hist(calendar['listing_id']);
# can't determine much from this distribution

In [ ]:
plt.hist(calendar['date']);
# not helpful at all

Filter down the listings dataframe. 

In [ ]:
# Drop columns not used in analysis:
    # text-heavy columns (not sure how to clean these up for this analysis)
    # URL columns
    # location columns: latitude, longitude, street, is_location_exact
    # columns with only one value: country (United States), country_code (US), state (MA), last_scraped (2016-09-07), 
        # calendar_last_scraped (2016-09-06), scrape_id (20160906204935), experiences_offered (None)
    # columns with similar info: city, zipcode, smart_location, neighbourhood; keep neighbourhood_cleansed
    # columns with only null values: jurisdiction_names, has_availability, neighbourhood_group_cleansed, license
print(listings[['country', 'country_code', 'state', 'market', 'last_scraped', 'calendar_last_scraped', 
                  'experiences_offered']].value_counts())
print(listings[['jurisdiction_names', 'has_availability', 'neighbourhood_group_cleansed', 
                   'license']].isnull().mean())

listings_filtered = listings.drop(['description', 'summary', 'space', 'neighborhood_overview', 'notes', 
                                   'transit', 'access', 'interaction', 'house_rules', 'host_about', 
                                   'name', 'host_name', 'host_verifications', 
                                   'host_thumbnail_url', 'listing_url', 'thumbnail_url', 'host_url', 
                                   'host_picture_url', 'medium_url', 'picture_url', 'xl_picture_url',
                                   'latitude', 'longitude', 'country', 'country_code', 'street', 'state', 
                                   'market', 'neighbourhood', 'smart_location', 'is_location_exact', 'city', 
                                   'zipcode', 'jurisdiction_names', 'last_scraped', 'calendar_last_scraped', 
                                   'scrape_id', 'experiences_offered', 'host_id', 'has_availability', 
                                   'neighbourhood_group_cleansed', 'license', 'host_location',
                                   'host_neighbourhood'], axis = 1)
listings_filtered.head() # confirm data looks as expected

In [ ]:
# Let's recheck:
# How many nulls? What proportion of nulls? What are the column types?
for column in listings_filtered.columns:
    num_null = listings_filtered[column].isnull().sum()
    prop_null = listings_filtered[column].isnull().mean()
    col_type = listings_filtered[column].dtype
    print(column + ":" + str(num_null) + "," + str(prop_null) + "," + str(col_type))

Clean up data types in the listings dataframe.

In [ ]:
# Why are price fields considered object type? Let's fix this.
listings_filtered[['price', 'extra_people', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee']].head()

In [ ]:
# price field has object dtype due to $ character
# Remove $ from beginning of each string
listings_filtered['weekly_price'] = listings_filtered['weekly_price'].str.replace("$", "")
listings_filtered['monthly_price'] = listings_filtered['monthly_price'].str.replace("$", "")
listings_filtered['security_deposit'] = listings_filtered['security_deposit'].str.replace("$", "")
listings_filtered['cleaning_fee'] = listings_filtered['cleaning_fee'].str.replace("$", "")
listings_filtered['extra_people'] = listings_filtered['extra_people'].str.replace("$", "")
listings_filtered['price'] = listings_filtered['price'].str.replace("$", "")

listings_filtered[['weekly_price', 'monthly_price', 'security_deposit', 
                   'cleaning_fee', 'extra_people', 'price']].head() # confirm changes worked correctly

In [ ]:
# Why are host_response_rate and host_acceptance_rate object type?
print(listings_filtered[['host_response_rate', 'host_acceptance_rate']].head())

# object dtype due to % character --> remove % from end of each string
listings_filtered['host_response_rate'] = listings_filtered['host_response_rate'].str.replace("%", "")
listings_filtered['host_acceptance_rate'] = listings_filtered['host_acceptance_rate'].str.replace("%", "")

print(listings_filtered[['host_response_rate', 'host_acceptance_rate']].head()) # confirm changes were successful

In [ ]:
# Convert price and fee fields to numeric type
listings_filtered['weekly_price'] = pd.to_numeric(listings_filtered['weekly_price'], errors= "coerce")
listings_filtered['monthly_price'] = pd.to_numeric(listings_filtered['monthly_price'], errors= "coerce")
listings_filtered['security_deposit'] = pd.to_numeric(listings_filtered['security_deposit'], errors= "coerce")
listings_filtered['cleaning_fee'] = pd.to_numeric(listings_filtered['cleaning_fee'], errors= "coerce")
listings_filtered['extra_people'] = pd.to_numeric(listings_filtered['extra_people'], errors= "coerce")
listings_filtered['price'] = pd.to_numeric(listings_filtered['price'], errors= "coerce")


# Convert host_response_rate and host_acceptance_rate to numeric type
listings_filtered['host_response_rate'] = pd.to_numeric(listings_filtered['host_response_rate'], errors= "coerce")
listings_filtered['host_acceptance_rate'] = pd.to_numeric(listings_filtered['host_acceptance_rate'], errors= "coerce")


listings_filtered.dtypes # confirm changes were successful

In [ ]:
# Post-filtering and cleaning: How many nulls? What proportion of nulls? What are the column types?
for column in listings_filtered.columns:
    num_null = listings_filtered[column].isnull().sum()
    prop_null = listings_filtered[column].isnull().mean()
    col_type = listings_filtered[column].dtype
    if num_null > 0:
        print(column + ":" + str(num_null) + "," + str(prop_null) + "," + str(col_type))
    else:
        continue

Section 4: Data Modeling (Optional)

Section 5: Evaluate the Results

Question 1: How does seasonal variation impact Boston AirBnB booking prices?

In [ ]:
# use calendar dataset
# extract month and year from 'date' column and store as new columns
calendar['month'] = calendar['date'].dt.month
calendar['year'] = calendar['date'].dt.year
calendar.head()

In [ ]:
calendar.isnull().mean()
# 51% of the price values are null

In [ ]:
# filter out available AirBnBs so that we're only looking at booked records
calendar_booked = calendar[calendar['available'] == 'f']
calendar_booked.head()

In [ ]:
calendar_booked.isnull().mean()
# 100% of price values are null for records where available = f
# this makes sense because the property is unavailable, so there is no price to rent

In [ ]:
calendar_available = calendar[calendar['available'] == 't']
calendar_available.head()

In [ ]:
calendar_available.isnull().mean()
# only 0.3% of price values are null for dates when the property is available
calendar_available.isnull().sum()


In [ ]:
# plot the average price by month
mean_monthly_price = calendar.groupby('month')['price'].mean().reset_index()
plt.plot(mean_monthly_price['month'], mean_monthly_price['price'], marker = 'o')
plt.title('Average Price by Month for a Boston AirBnB')
plt.xlabel('Month')
plt.ylabel('Average Price')
plt.xticks(np.arange(1, 13, 1))
plt.tight_layout()
plt.show()
# highest average prices are in september and october, respectively

Question 2: What are the most common amenities for Boston AirBnB properties?

In [ ]:
freq = []
for s in listings_filtered['amenities']:
    new_list = s[1:-1].replace('"', '').split(',')
    freq = freq + new_list

df = pd.DataFrame({'data': freq})
freq_table = df['data'].value_counts()
amenities = pd.DataFrame(freq_table)
amenities

In [475]:
amenities_clean = amenities.drop(['', 'translation missing: en.hosting_amenity_50', 'translation missing: en.hosting_amenity_49'], axis=0)

In [ ]:
plt.figure(figsize=(8, 6))
ax = sns.barplot(x='data', y='count', data=amenities_clean)
plt.xlabel('Amenities')
plt.ylabel('Number of Properties')
plt.title('Boston AirBnB Amenities')
plt.xticks(rotation = 90)

ax2=ax.twinx()
ax2.set_ylabel('Frequency of Properties')

plt.show()

Question 3: Which neighborhood has the most AirBnB properties and how does that correlate to pricing?

In [ ]:
neighborhood_counts = listings_filtered.groupby(by='neighbourhood_cleansed')['id'].nunique().sort_values()
neighborhood_counts

In [ ]:
mean_neighborhood = listings_filtered.groupby(by='neighbourhood_cleansed').mean('price').sort_values(by='price', ascending=False)
mean_neighborhood

In [ ]:
plt.figure(figsize=(8, 6))
ax = sns.barplot(x='neighbourhood_cleansed', y='price', data=mean_neighborhood)
plt.xlabel('Neighbourhood')
plt.ylabel('Price')
plt.title('Boston AirBnB Average Prices by Neighbourhood')
plt.xticks(rotation = 90)

plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(neighborhood_counts, mean_neighborhood['price'])
plt.ylim(bottom=0, top=350)
plt.xlabel('Number of Properties per Neighborhood')
plt.ylabel('Average Price')
plt.title('Boston AirBnB Average Prices by Number of Properties in Neighborhood')